# Installing libraries and dependencies

In [ ]:
import nltk
import spacy
!pip install squarify
nlp = spacy.load('en_core_web_sm')
nltk.download("stopwords")
nltk.download('popular')
# Install nltk Dependencies
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('data')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Importing libraries and dependencies

In [ ]:
import nltk
from datetime import datetime
import spacy
import os
from os import listdir
import seaborn as sns
import csv
from os.path import isfile, join
from io import StringIO
import re
import string
import csv
import gensim
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from gensim.models.phrases import Phraser, Phrases
from spacy.matcher import Matcher, PhraseMatcher
from nltk.util import ngrams
nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

# Model

**Creating word embeddings**

In [ ]:
from gensim.models import Word2Vec
import string
import re
from nltk.corpus import stopwords
with open(r'skills.txt',encoding="utf8") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [ ]:
x=[]
for line in content:
    tokens=word_tokenize(line)
    tok=[w.lower() for w in tokens]
    table=str.maketrans('','',string.punctuation)
    strpp=[w.translate(table) for w in tok]
    words=[word for word in strpp if word.isalpha()]
    stop_words=set(stopwords.words('english'))
    words=[w for w in words if not w in stop_words]
    x.append(words)

In [ ]:
texts=x

**Removing words that are not useful**

In [ ]:
with open('common.txt') as f:
    content2 = f.read()
ntexts=[]
l=len(texts)
for j in range(l):
    s=texts[j]
    res = [i for i in s if i not in content2]
    ntexts.append(res)
print(texts[6])

In [ ]:
print(len(ntexts))
texts=ntexts

In [ ]:
content=texts

In [ ]:
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
x=ntexts
# Create the relevant phrases from the list of sentences:
phrases = Phrases(x, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)
# Applying the Phraser to transform our sentences
all_sentences = list(bigram[x])
model=gensim.models.Word2Vec(all_sentences,size=5000,min_count=2,workers=4,window=4)
model.save("final.model")
wrds=list(model.wv.vocab)
print(len(wrds))

In [ ]:
a=model.wv.most_similar("machine_learning")
b=model.wv.most_similar("analytics")
c=model.wv.most_similar("ai")

In [ ]:
print(a)
print(b)
print(c)

**ADDING new words in the vocab of pre-trained model**

In [ ]:
model = Word2Vec.load("final.model") 
model.min_count = 1
docs = [["Oracle", "MERN"]]
model.build_vocab(docs, update=True)
model.train(docs, total_examples = len(docs), epochs = 10)
#model.train([["Oracle", "MERN"]], total_examples=1, epochs=1)

In [ ]:
print(len(model.wv.vocab))

In [ ]:
d=model.wv.most_similar("Oracle")
d

# **RESUME**

**Stopwords Section**

In [ ]:
nlp_stopwords = stopwords.words('english')
gensim_stopwords = gensim.parsing.preprocessing.STOPWORDS
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words

stopwords_list = stopwords.words('english')

sw_list = ['summary','year','education','page','summary','college','university','company','skills','bachelor','country',
           'address','email','name','phone','house','jan','feb','mar','apr','may','june','july','aug','sep','oct','nov',
           'dec','com','gmail','school','hospital']

stopwords_list.extend(gensim_stopwords) #appending gensim stopwords in nltk
stopwords_list.extend(spacy_stopwords) #appending spacy stopwords in nltk
stopwords_list.extend(sw_list) #appending customized stopwords in nltk

**Function for cleaning data**

In [ ]:
def clean_data(file_content):

    file_content = file_content.lower()
    file_content = re.sub('\[.*?\]', '', file_content)
    file_content = re.sub('[‘’“”…]', '', file_content)
    file_content = re.sub('\n', '', file_content) 
    file_content = re.sub('\t\t', '', file_content)
    file_content = re.sub('[%s]' % re.escape(string.punctuation), '', file_content)
    file_content = re.sub('\w*\d\w*', '', file_content)
    file_content = re.sub('\W', ' ', file_content)
    file_content = remove_stopwords(file_content)
    tokens = word_tokenize(file_content)
    tokens_without_sw = [word for word in tokens if not word in stopwords_list]
    lemmatizer = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokens_without_sw])
    s = list(str(lemmatized_output))
    r  = ''.join([lemmatizer.lemmatize(r) for r in s])
    tokens2 = word_tokenize(r)
    return tokens2

##### **Function for extracting email**

In [ ]:
def extract_email(text):
    email = re.compile(r'[a-zA-Z0-9-\.]+@[a-zA-Z-\.]*\.(com|edu|net)')
    matches = email.finditer(text)
    for match in matches:
        return(match.group(0))

**Function for extracting Phone**

In [ ]:
def extract_phone(text):
    phone = re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?')
    matches = phone.finditer(text)
    for match in matches:
        return(match.group(0))

**Function for extracting Name**

In [ ]:
def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', [pattern]) #matching
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

**Function for extracting technical skills**

In [ ]:
def extract_skills(resume_text):
    nlp_text = nlp(resume_text)
    tokens = [token.text for token in nlp_text if not token.is_stop]
    data = pd.read_csv("skills.csv") 
    skills = list(data.columns.values)
    skillset = []
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

**Function for extracting education**

In [ ]:
# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII',
            'Matriculation','BSC','PhD'
        ]
def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

**Function for extracting analytical skills**

In [ ]:
def extract_analytical_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("AnalyticalSkills.csv") 
    
    # extract values
    a_skills = list(data.columns.values)
    
    a_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in a_skills:
            a_skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in a_skillset])]

**Function for extracting Leadership Skills**

In [ ]:
def extract_leadership_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("LeadershipSkills.csv") 
    
    # extract values
    l_skills = list(data.columns.values)
    
    l_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in l_skills:
            l_skillset.append(token)

    return [i.capitalize() for i in set([i.lower() for i in l_skillset])]

**Function for extracting Teamwork skills**

In [ ]:
def extract_teamwork_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("TeamworkSkills.csv") 
    
    # extract values
    t_skills = list(data.columns.values)
    
    t_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in t_skills:
            t_skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in t_skillset])]

**Function for extracting Communication skills**

In [ ]:
def extract_communication_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("CommunicationSkills.csv") 
    
    # extract values
    c_skills = list(data.columns.values)
    
    c_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in c_skills:
            c_skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in c_skillset])]

**Function for extracting action verbs**

In [ ]:
def extract_action_verbs(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("Action_words.csv") 
    
    # extract values
    c_skills = list(data.columns.values)
    
    c_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in c_skills:
            c_skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in c_skillset])]

**Function for extracting measureable results**

In [ ]:
def extract_measureable_results(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("Measurable_results.csv") 
    
    # extract values
    c_skills = list(data.columns.values)
    
    c_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in c_skills:
            c_skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in c_skillset])]

**Function for extracting result-driven skills**

In [ ]:
def extract_result_driven_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("Result_driven_skills.csv") 
    
    # extract values
    c_skills = list(data.columns.values)
    
    c_skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in c_skills:
            c_skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in c_skillset])]

**Function for extracting Strengths**

In [ ]:
def extract_strength(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("combined.csv") 
    
    # extract values
    strengths = list(data.columns.values)
    
    strengthsset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in strengths:
            strengthsset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in strengthsset])]

**Function for cleaning through nlp**

In [ ]:
def nlp_cleaning(content, file_content):
    digits = []
    name_nlp = []
    loc = []
    date = []
    norp = []
    law = []
    percent = []
    product = []
    time = []
    x = []
    
    doc = nlp(content)
    for ent in doc.ents:
    #print(f'Entity:{ent}, Label: {ent.label_}')
        if 'ORG' in ent.label_:
            x.append(ent)
        if 'CARDINAL' in ent.label_:
            digits.append(ent)
        if 'PERSON' in ent.label_:
            name_nlp.append(ent)
        if 'GPE' in ent.label_:
            loc.append(ent)
        if 'DATE' in ent.label_:
            date.append(ent)
        if 'NORP' in ent.label_:
            norp.append(ent)
        if 'LAW' in ent.label_:
            law.append(ent)
        if 'PERCENT' in ent.label_:
            percent.append(ent)
        if 'PRODUCT' in ent.label_:
            product.append(ent)
        if 'TIME' in ent.label_:
            time.append(ent)
    x = str(x)
    x_tokens = word_tokenize(x)
    date = str(date)
    date_tokens = word_tokenize(date)
    loc = str(loc)
    loc_tokens = word_tokenize(loc)
    digits = str(digits)
    digits_tokens = word_tokenize(digits)
    norp = str(norp)
    norp_tokens = word_tokenize(norp)
    law = str(law)
    law_tokens = word_tokenize(law)
    percent = str(percent)
    percent_tokens = word_tokenize(percent)
    product = str(product)
    product_tokens = word_tokenize(product)
    time = str(time)
    time_tokens = word_tokenize(time)
    
    mylist = [word for word in file_content if word not in date_tokens and not word in x_tokens and not word in loc_tokens and not word in digits_tokens 
              and word not in norp_tokens and not word in law_tokens and not word in percent_tokens and not word in 
              product_tokens and word not in time_tokens]
    
    lemmatizer = WordNetLemmatizer()
    final = ' '.join([lemmatizer.lemmatize(w) for w in mylist])
    return final

**Function for creating wordcloud**

In [ ]:
def create_wordcloud(final):
    wordcloud = WordCloud(max_font_size=35, max_words=100, background_color="white").generate(final)
    plt.figure(figsize = (5,5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

**Function for creating wordcloud of skills**

In [ ]:
def create_wordcloud_of_skills(skills):
    cloud = WordCloud(max_font_size=80,colormap="hsv").generate(skills)
    plt.figure(figsize=(12,12))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

**Function for Experience**

In [ ]:
def experience_finder(c):
    all_ex=[]
    month=["January","February","March","April","May","June","July","August","September","October","November","December"]
    for i in c:
        start_date=i[0].split()
        
        if start_date[0] in month:
            index=month.index(start_date[0])
            index+=1
        start_date = datetime(int(start_date[-1]),index,1,0, 0, 0)
        end_date=i[3].split()
        
        if end_date[0] in month:
            index=month.index(end_date[0])
            index+=1
        
        end_date = datetime(int(end_date[-1]),index,1,0, 0, 0)
        experince=end_date-start_date
        if float(str(experince).split(",")[0].split()[0])<365:
            experince=float(str(experince).split(",")[0].split()[0])
            experince=experince/365
            experince*=12
            experince=str(experince)+ " months"
        elif float(str(experince).split(",")[0].split()[0])>=365:    
            experince=experince/365
            experince=str(experince).split(",")[0].split()[0]+ " years"
        all_ex.append(experince)
    
    return all_ex

**Reading Reseume from directory**

In [ ]:
mypath=r'Resumes'
#Path for the files
onlyfiles = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [ ]:
len(onlyfiles)

In [ ]:
onlyfiles

**Function for creating Personal Deatils csv**

In [ ]:
header = ['Name','Email','Phone','Education','Expereince','Technical Skills','Analytical Skills','Leadership Skills',
          'Teamwork Skills','Communication Skills','Action verbs']    

with open('Personal Details.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

**Function for creating csv for Visualization**

In [ ]:
header = ['Name','Expereince','Analytical Skills','Leadership Skills','Teamwork Skills','Communication Skills','Action verbs',
          'Measureable Results','Result-driven Skills','Strengths','Soft Skills']    

with open('Details.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

# Information Extraction

In [ ]:
f = []
list1 = []


def main(file):
    txt_file = open(file,'rt',encoding = 'utf-8')
    content = txt_file.read()
    file_content = content
    #######################################################################
    date = []
    doc = nlp(content)
    exp = []
    
    for ent in doc.ents:
        if 'DATE' in ent.label_:
            date.append(ent)
    m = str(date)
    try: 
        c = re.findall(r"(?P<fmonth>\w+(?P<fh>.)\d+)\s*(\s|-|to)\s*(?P<smonth>\w+(?P<sh>.)\d+|present|date|now)",m)
        all_experience=experience_finder(c)
        count=0
        for x in all_experience:
            if "years" in x:
                x=float(x.split()[0])
                count=count+x
            elif 'months' in x:
                x=float(x.split()[0])/12
                count=count+x
                exp = count

    except:
        exp = 0
    ######################################################################
    
    final_list = [] 
    details = []
    
    name = extract_name(file_content)
    print("Name : ",name)
    
    email = extract_email(file_content)
    print("\nEmail : ",email)
    
    phone = extract_phone(file_content)
    print("\nPhone : ",phone)
    
    technical_skills = extract_skills(file_content)
    print("\nTechincal Skills : ",technical_skills)
    
    analytical_skills = extract_analytical_skills(file_content)
    a = len(analytical_skills)
    print("\nAnalytical Skills : ",analytical_skills)
    
    try:
        
        listToStr = ' '.join([str(elem) for elem in analytical_skills])
        print("\n**************************************** WORLCLOUD FOR ANALYTICAL SKILLS ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No analytical skill")
    
    leadership_skills = extract_leadership_skills(file_content)
    b = len(leadership_skills)
    print(" \nLeadership Skills : ",leadership_skills)
    
    try: 
        listToStr = ' '.join([str(elem) for elem in leadership_skills])
        print("\n**************************************** WORLCLOUD FOR LEADERSHIP SKILLS ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No leadership skill")
    
    teamwork_skills = extract_teamwork_skills(file_content)
    c = len(teamwork_skills)
    print(" \nTeamwork Skills : ",teamwork_skills)
    
    try: 
        listToStr = ' '.join([str(elem) for elem in teamwork_skills])
        print("\n**************************************** WORLCLOUD FOR TEAMWORK SKILLS ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No Teamwork skill")
    
    communication_skills = extract_communication_skills(file_content)
    d = len(communication_skills)
    print(" \nCommunication Skills : ",communication_skills)
    
    try: 
        listToStr = ' '.join([str(elem) for elem in communication_skills])
        print("\n***************************************** WORLCLOUD FOR COMMUNICATION SKILLS ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No communication skill")
    
    action_verbs = extract_action_verbs(file_content)
    e = len(action_verbs)
    print(" \nAction Verbs :  ",action_verbs)
    
    
    try: 
        listToStr = ' '.join([str(elem) for elem in action_verbs])
        print("\n***************************************** WORLCLOUD FOR ACTION VERBS ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No action verbs")
    
    measureable_result = extract_measureable_results(file_content)
    f = len(measureable_result)
    print(" \nMeasureable Results :  ",measureable_result)
    try:
        
        listToStr = ' '.join([str(elem) for elem in measureable_result])
        print("\n**************************************** WORLCLOUD FOR MEASUREABLE RESULT ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No measureable result")
    
    result_driven_skills = extract_result_driven_skills(file_content)
    g = len(result_driven_skills)
    print(" \nResult driven skills :  ",result_driven_skills)
    
    
    try:
        listToStr = ' '.join([str(elem) for elem in result_driven_skills])
        print("\n**************************************** WORLCLOUD FOR RESULT DRIVEN SKILLS ****************************************")
        wordcloud = create_wordcloud(listToStr)
    except:
        print("No result driven skill")
    
    strength = extract_strength(file_content)
    print(" \nStrength : ",strength)
    
    education = extract_education(file_content)
    print(" \nEducation : ",education)
    
    print("\nExperience",exp)
    
    strengths_skills = strength + technical_skills
    h = len(strengths_skills)
    
    soft_skills = analytical_skills + leadership_skills + teamwork_skills + communication_skills + action_verbs + measureable_result + result_driven_skills  
    i = len(soft_skills)
    
    final_list.append(name)
    final_list.append(email)
    final_list.append(phone)
    final_list.append(education)
    final_list.append(exp)
    final_list.append(technical_skills)
    final_list.append(analytical_skills)
    final_list.append(leadership_skills)
    final_list.append(teamwork_skills)
    final_list.append(communication_skills)
    final_list.append(action_verbs)
    
    details.append(name)
    details.append(exp)
    details.append(a)
    details.append(b)
    details.append(c)
    details.append(d)
    details.append(e)
    details.append(f)
    details.append(g)
    details.append(h)
    details.append(i)

   
    with open(r'Personal Details.csv', 'a+', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(final_list)
    
    with open(r'Details.csv', 'a+', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(details)
    
    file_content = clean_data(file_content)
    file_content = nlp_cleaning(content,file_content)

    
    print("\n***************************************** WORLCLOUD FOR RESUME ****************************************")
    wordcloud = create_wordcloud(file_content)
    list1 = [file_content]
    
    print("\n**************************************** WORLCLOUD FOR STRENGTH ****************************************")
    listToStr = ' '.join([str(elem) for elem in strengths_skills])
    wordcloud = create_wordcloud(listToStr)
    
    print("\n**************************************** WORLCLOUD FOR SOFT SKILLS ****************************************")
    listToStr = ' '.join([str(elem) for elem in soft_skills])
    try : 
        wordcloud = create_wordcloud_of_skills(listToStr)
    except:
        print(" ")
    
    print("\n******************************************* SOFT SKILLS CLASSIFICATION **************************************\n")
    
    labels = ['Analytical Skills','Leadership Skills','Teamwork Skills','Communication Skills']
    data1 = [a,b,c,d]
    colors = ['lightskyblue','greenyellow','hotpink','gold']
    fig1, ax1 = plt.subplots()
    plt.pie(data1, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
    #draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Equal aspect ratio ensures that pie is drawn as a circle
    ax1.axis('equal')  
    plt.tight_layout()
    plt.show()
    
    print("\n******************************************* SOFT SKILLS vs TECHNICAL SKILLS **************************************\n")
    p = len(soft_skills)
    q = len(strengths_skills)
    labels = ['Soft Skills','Technical Skills']
    data2 = [p,q]
    colors = ['lightskyblue','greenyellow']
    fig1, ax1 = plt.subplots()
    plt.pie(data2, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
    #draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Equal aspect ratio ensures that pie is drawn as a circle
    ax1.axis('equal')  
    plt.tight_layout()
    plt.show()

# CREATING PROFILE

In [ ]:
def create_profile(file):
    model=Word2Vec.load("final.model")
    txt_file = open(file,'rt',encoding = 'utf-8')
    text = txt_file.read()
    text = text.lower()
    
    keywords = ['statistics','language','machine_learning','deep','artificial_intelligence','python','data']
    
    for i in range(len(keywords)):
        word=keywords[i]
        x = [nlp(text[0]) for text in model.wv.most_similar(word)]

    print("*******************************************")
    
    #print(stats_words,NLP_words)
    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('KEYWORD', None, *x)

    doc = nlp(text)
    
    d = []  
    matches = matcher(doc)
    for match_id, start, end in matches:
        rule_id = nlp.vocab.strings[match_id]  # get the unicode I
        span = doc[start : end]               # get the matched slice of the doc
        d.append((rule_id, span.text))      
    keywords = "\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())
    print("KEYWORDS")
    print(keywords)
    
    ## convertimg string of keywords to dataframe
    df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
    df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
    df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
    df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
    df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
    print("********************DF********************")
    print(df)
    
    base = os.path.basename(file)
    filename = os.path.splitext(base)[0]
    
       
    name = filename.split('_')
    print(name)
    name2 = name[0]
    name2 = name2.lower()
    ## converting str to dataframe
    name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])
    
    dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)
    print("******************DATAF**************")
    print(dataf)

    return(dataf)

# MAIN FUNCTION

In [ ]:
#Code to execute the above functions 
final_db=pd.DataFrame()
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    data=main(file)
    print(data)
    dat=create_profile(file)
    final_db=final_db.append(dat)
    i+=1

In [ ]:
#Code to count words under each category and visualize it through MAtplotlib
final_db2 = final_db['Keyword'].groupby([final_db['Candidate Name'], final_db['Subject']]).count().unstack()
final_db2.reset_index(inplace = True)
final_db2.fillna(0,inplace=True)
candidate_data = final_db2.iloc[:,1:]
candidate_data.index = final_db2['Candidate Name']
#the candidate profile in a csv format
cand=candidate_data.to_csv('candidate_profile.csv')
cand_profile=pd.read_csv('candidate_profile.csv')
cand_profile

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 25})
ax = candidate_data.plot.barh(title="Keywords in Resume according to category", legend=False, figsize=(50,60),stacked=True)
skills = []
for j in candidate_data.columns:
    for i in candidate_data.index:
        skill = str(j)+": " + str(candidate_data.loc[i][j])
        skills.append(skill)
patches = ax.patches
for skill, rect in zip(skills, patches):
    width = rect.get_width()
    if width > 0:
        x = rect.get_x()
        y = rect.get_y()
        height = rect.get_height()
        ax.text(x + width/2., y + height/2., skill, ha='center', va='center')
plt.show()

# HEATMAP

In [ ]:
final_db['Keyword'].groupby([final_db['Candidate Name'], final_db['Subject']]).count().unstack()

In [ ]:
data = (
    final_db
    .groupby(final_db['Candidate Name'])
    .Subject
    .value_counts()
    .unstack()
    .fillna(0)
)
sns.set_theme()

In [ ]:
plt.figure(dpi=1800)
sns.heatmap(data,linewidths=.12)

# VISUALIZING SKILLS OF EMLOYEES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df2 = pd.read_csv("Details.csv")

In [ ]:
df2.drop(['Expereince','Soft Skills'],inplace = True,axis =1)

In [ ]:
import numpy as np

In [ ]:
colors = ['red','gold','lightskyblue','hotpink','greenyellow','orange','yellow','lightcoral']
df2.plot(
  x = 'Name', 
  kind = 'barh', 
  color = colors,
  stacked = True, 
  figsize = (25,35),
  title = 'Soft Skills Percentage of Employees', 
  mark_right = True)

df_total = df2['Strengths'] + df2["Analytical Skills"] + df2["Leadership Skills"] + df2["Teamwork Skills"] + df2["Communication Skills"] + df2["Action verbs"] + df2["Measureable Results"] + df2["Result-driven Skills"]
df_rel = df2[df2.columns[1:]].div(df_total, 0)*100

for n in df_rel:
    for i, (cs, ab, pc) in enumerate(zip(df2.iloc[:, 1:].cumsum(1)[n], df2[n], df_rel[n])):

        plt.text(cs - ab / 2, i, str(np.round(pc, 1)) + '%', va = 'center', ha = 'center')

# COMPARING SOFT SKILLS vs TECHNICAL SKILLS of EMPLOYEES

In [ ]:
df3 = pd.read_csv("Details.csv")

In [ ]:
df3.drop(['Expereince','Analytical Skills','Leadership Skills','Communication Skills','Teamwork Skills','Action verbs','Measureable Results','Result-driven Skills'],inplace = True,axis =1)

In [ ]:
colors = ['lightskyblue','hotpink']
df3.plot(
  x = 'Name', 
  kind = 'barh', 
  color = colors,
  stacked = True, 
  figsize = (25,35),
  title = 'Soft Skills vs Technical Strengths of Employees', 
  mark_right = True)

df_total = df3['Strengths'] + df3["Soft Skills"]
df_rel = df3[df3.columns[1:]].div(df_total, 0)*100

for n in df_rel:
    for i, (cs, ab, pc) in enumerate(zip(df3.iloc[:, 1:].cumsum(1)[n], df3[n], df_rel[n])):

        plt.text(cs - ab / 2, i, str(np.round(pc, 1)) + '%', va = 'center', ha = 'center')

# EDUCATION VISUALIZATION

In [ ]:
x = pd.read_csv("Personal Details.csv")

In [ ]:
# Compute percentages from csv
degrees = set(x['Education'])
percentages = []
for degree in degrees:
    percentages.append(x[x['Education'] == degree].shape[0])
percentages = np.array(percentages)
percentages = ((percentages / percentages.sum()) * 100)

In [ ]:
# Create labels for tree map
labels = [degree + '\n({0:.1f}%)'.format(percentage) for degree, percentage in zip(degrees, percentages)]

In [ ]:
%matplotlib inline
import squarify
sns.set()
# Create figure
plt.figure(figsize=(25, 25), dpi=500)
squarify.plot(percentages, label=labels, color=sns.color_palette('colorblind', len(degrees)))
plt.axis('off')
# Add title
plt.title('Degrees')
# Show plot
plt.show()

# SKILLS SCORING

In [ ]:
df2 = pd.read_csv("Details.csv")

In [ ]:
group_analyticalskills = df2[df2.columns[2]].tolist()
group_leadershipskills = df2[df2.columns[3]].tolist()
group_teamworkskills = df2[df2.columns[4]].tolist()
group_communicationskills = df2[df2.columns[5]].tolist()
group_actionverbs = df2[df2.columns[6]].tolist()
group_measureableresults = df2[df2.columns[7]].tolist()
group_resultdrivenskills = df2[df2.columns[8]].tolist()
group_strengths = df2[df2.columns[9]].tolist()

In [ ]:
data = pd.DataFrame({'Skills': ['Analytical'] * len(group_analyticalskills) + ['Leadership'] * len(group_leadershipskills)
                    + ['Teamwork'] * len(group_teamworkskills) + ['Communication'] * len(group_communicationskills) + 
                    ['Action verbs'] * len(group_actionverbs) + ['Measureable Results'] * len(group_measureableresults)
                    + ['Result-driven'] * len(group_resultdrivenskills) + ['Technical'] * len(group_strengths),
                    'Skills score': group_analyticalskills + group_leadershipskills + group_teamworkskills + group_communicationskills + 
                     group_actionverbs + group_measureableresults + group_resultdrivenskills + group_strengths})

In [ ]:
# Create figure
plt.figure(dpi=250)
plt.rc('xtick', labelsize=5)
# Set style
sns.set_style('whitegrid')
# Create boxplot
sns.violinplot('Skills', 'Skills score', data=data)
# Despine
sns.despine(left=True, right=True, top=True)
# Add title
plt.title('Skills scores')
# Show plot
plt.show()

# EMPLOYEES EXPERIENCE

In [ ]:
df3 = pd.read_csv("Details.csv")

In [ ]:
df3['Expereince'].replace({"[]": "0"}, inplace=True)

In [ ]:
m = df3['Name'].tolist()
n = df3['Expereince'].tolist()

In [ ]:
%matplotlib inline
plt.figure(figsize = (10, 15))
plt.barh(m,n,color = 'purple')
plt.xlabel('Expereince')
plt.ylabel('Candidate')
plt.show()

In [ ]:
import pandas as pd
df4 = pd.read_csv("Details.csv")

In [ ]:
p = df4['Soft Skills'] + df4['Strengths']
p

In [ ]:
import matplotlib.pyplot as plt
# Create figure
plt.figure(figsize=(6, 4), dpi=150)
# Create histogram
plt.hist(p, bins=10)
plt.axvline(x=50, color='r')
plt.axvline(x=100, color='r', linestyle= '--')
plt.axvline(x=150, color='r', linestyle= '--')
plt.axvline(x=200, color='r', linestyle= '--')
# Add labels and title
plt.xlabel('Skill score')
plt.ylabel('Frequency')
plt.title('Skills scores for candidates')
# Show plot
plt.show()